# データの読み込み

In [1]:
!mmseqs easy-linclust /home/tamon/Desktop/ProtGPT2/MDH_37000.fasta MDH_mmseqs tmp --min-seq-id 0.8

easy-linclust /home/tamon/Desktop/ProtGPT2/MDH_37000.fasta MDH_mmseqs tmp --min-seq-id 0.8 

MMseqs Version:                     	13.45111
Cluster mode                        	0
Max connected component depth       	1000
Similarity type                     	2
Threads                             	128
Compressed                          	0
Verbosity                           	3
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                       	false
Alignment mode                      	0
Alignment mode                      	0
Allow wrapped scoring               	false
E-value threshold                   	0.001
Seq. id. threshold                  	0.8
Min alignment length                	0
Seq. id. mode                       	0
Alternative alignments              	0
Coverage threshold                  	0.8
Coverage mode                       	0
Max sequence length                 	65535
Compositional bias                  	1
Max reject            

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Time for processing: 0h 0m 0s 67ms
rmdb tmp/9333942286420908362/input -v 3 

Time for processing: 0h 0m 0s 1ms
rmdb tmp/9333942286420908362/input_h -v 3 

Time for processing: 0h 0m 0s 0ms
rmdb tmp/9333942286420908362/clu_seqs -v 3 

Time for processing: 0h 0m 0s 4ms
rmdb tmp/9333942286420908362/clu_rep -v 3 

Time for processing: 0h 0m 0s 0ms
rmdb tmp/9333942286420908362/clu -v 3 

Time for processing: 0h 0m 0s 2ms


In [ ]:
#上のデータはUnreviewdのものだけなのでReviewedのデータも結合しておく

In [2]:
from Bio import SeqIO
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline

# mmseq後のFASTAファイルのパス
fasta_file = "/home/tamon/Desktop/ProtGPT2/MDH_mmseqs_rep_seq.fasta"

# データを格納するリスト
headers = []
sequences = []

# FASTAファイルをパースしてデータを取得
for record in SeqIO.parse(fasta_file, "fasta"):
    headers.append( "<|endoftext|>")
    sequences.append(str(record.seq))

# データフレームの作成
df = pd.DataFrame({"Header": headers, "Sequence": sequences})

# データフレームの表示
df

,Header,Sequence
0,<|endoftext|>,MKQPVHIAVTGAAGQISYSLLFRLAAGALLGPEQPIILHLLEIKPA...
1,<|endoftext|>,MKRKKIAVVGGGQIGGILALLVAQKELGDLVILDIPDKENFIKGKA...
2,<|endoftext|>,MSRKKIALIGGGQIGGILALLSTQKSLGDIVIVDLPQLEDPMKGKA...
3,<|endoftext|>,MAIRKITIVGAGRVGEATAQFLAKNELCRELVLLDVQEGVAQGGAL...
4,<|endoftext|>,MSLRRKKITVVGAGFVGATAAHWAAAKELGDVCIVDIVEGLPQGKA...
...,...,...
9189,<|endoftext|>,MARSKIALIGAGQIGGTLAHLAGLKELGDVVLFDIADGVPQGKGLD...
9190,<|endoftext|>,MARKKIALIGGGQIGGTLAHLAGLKELGDIVIFDIVEGLPQGKALD...
9191,<|endoftext|>,MARKKIALIGAGMIGGTLAHLAAIKEMGDIVLVDVAEGIPEGKALD...
9192,<|endoftext|>,MPKAVVLGAAGGIGQPLALLLKANPLVTELALFDIVNTPGVAADLS...


In [3]:
#基本アミノ酸以外を含んだものを消去
amino_acid = ['A','R','N','D','C','E','Q','G','H','I','L','K','M','F','P','S','T','W','Y','V']

def Del_amino_acid(x):
    seq = x['Sequence']
    seq_key = pd.Series(list(seq)).value_counts().keys()
    seq_key = list(seq_key)
    for i in seq_key:
        if i not in amino_acid:
            seq = 0
    return seq

df['Sequence'] = df.apply(Del_amino_acid, axis=1)
df = df[df['Sequence']!=0].reset_index(drop=True)

print(len(df))

9034


In [4]:
df['length'] = df['Sequence'].map(lambda x:len(x))
df

,Header,Sequence,length
0,<|endoftext|>,MKQPVHIAVTGAAGQISYSLLFRLAAGALLGPEQPIILHLLEIKPA...,326
1,<|endoftext|>,MKRKKIAVVGGGQIGGILALLVAQKELGDLVILDIPDKENFIKGKA...,311
2,<|endoftext|>,MSRKKIALIGGGQIGGILALLSTQKSLGDIVIVDLPQLEDPMKGKA...,311
3,<|endoftext|>,MAIRKITIVGAGRVGEATAQFLAKNELCRELVLLDVQEGVAQGGAL...,311
4,<|endoftext|>,MSLRRKKITVVGAGFVGATAAHWAAAKELGDVCIVDIVEGLPQGKA...,311
...,...,...,...
9029,<|endoftext|>,MARSKIALIGAGQIGGTLAHLAGLKELGDVVLFDIADGVPQGKGLD...,320
9030,<|endoftext|>,MARKKIALIGGGQIGGTLAHLAGLKELGDIVIFDIVEGLPQGKALD...,320
9031,<|endoftext|>,MARKKIALIGAGMIGGTLAHLAAIKEMGDIVLVDVAEGIPEGKALD...,320
9032,<|endoftext|>,MPKAVVLGAAGGIGQPLALLLKANPLVTELALFDIVNTPGVAADLS...,335


In [5]:
df['length'].describe()

count    9034.000000
mean      330.207106
std        82.698603
min       104.000000
25%       311.000000
50%       324.000000
75%       332.000000
max      3476.000000
Name: length, dtype: float64

In [6]:
import textwrap
def Wrap(x):
    seq = "\n".join(textwrap.wrap(x['Sequence'], width=60))
    return seq

Sequence = df.apply(Wrap,axis=1)
Sequence

0       MKQPVHIAVTGAAGQISYSLLFRLAAGALLGPEQPIILHLLEIKPA...
1       MKRKKIAVVGGGQIGGILALLVAQKELGDLVILDIPDKENFIKGKA...
2       MSRKKIALIGGGQIGGILALLSTQKSLGDIVIVDLPQLEDPMKGKA...
3       MAIRKITIVGAGRVGEATAQFLAKNELCRELVLLDVQEGVAQGGAL...
4       MSLRRKKITVVGAGFVGATAAHWAAAKELGDVCIVDIVEGLPQGKA...
                              ...                        
9029    MARSKIALIGAGQIGGTLAHLAGLKELGDVVLFDIADGVPQGKGLD...
9030    MARKKIALIGGGQIGGTLAHLAGLKELGDIVIFDIVEGLPQGKALD...
9031    MARKKIALIGAGMIGGTLAHLAAIKEMGDIVLVDVAEGIPEGKALD...
9032    MPKAVVLGAAGGIGQPLALLLKANPLVTELALFDIVNTPGVAADLS...
9033    MDGMDIGSKGVVRVVVTGAAGQIGYSLVPLIAGGKMLGPGVKVELR...
Length: 9034, dtype: object

In [8]:
#解析用にcsvを出力
Sequence.to_csv('/home/tamon/Desktop/ProtGPT2/sequence.csv', index=False)

In [9]:
# データフレームの作成
df_2 = pd.DataFrame({"Header": '<|endoftext|>', "Sequence": Sequence})
vali = int(len(df_2)*0.2)
df_valid = df_2.sample(n=vali, random_state=0)
df_train = df_2.drop(df_valid.index)
df_valid = df_valid.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

print('評価用データ：',len(df_valid))
print('学習用データ：',len(df_train))

評価用データ： 1806
学習用データ： 7228


In [13]:
#テキストファイルでの出力
df_valid.to_csv('/home/tamon/Desktop/ProtGPT2/df_valid_MDH_90.txt', index=False, sep='\t')
df_train.to_csv('/home/tamon/Desktop/ProtGPT2/df_train_MDH_90.txt', index=False, sep='\t')